In [ ]:
# 파이썬 ≥3.5 필수
import sys
assert sys.version_info >= (3, 5)

# 사이킷런 ≥0.20 필수
import sklearn
assert sklearn.__version__ >= "0.20"

# 공통 모듈 임포트
import numpy as np
import os

# 노트북 실행 결과를 동일하게 유지하기 위해
np.random.seed(42)

# 깔끔한 그래프 출력을 위해
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# 그림을 저장할 위치
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "unsupervised_learning"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("그림 저장:", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# 불필요한 경고를 무시합니다 (사이파이 이슈 #5998 참조)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

우리가 사용할 수 있는 데이터는 대부분 레이블이 없다.
<br> 데이터에 레이블을 부여하기 위해서는 사람이 수동으로 처리해야하는데, 이는 매우 오래걸릴 뿐더러 비용이 많이 든다.
<br> 
<br> 사람이 모든 사진에 레이블을 붙일 필요 없이,
<br> **알고리즘이 레이블이 없는 데이터를 바로 사용하는 것이 비지도 학습(unsupervised machine learning)**이다.

비지도 학습 방법
* 차원 축소(dimensionality reduction)
* 군집(clustering)
* 이상치 탐지(outlier detection)
* 밀도 추정(density estimation)

#9.1 군집

> 군집이란?
<br>비슷한 샘플을 구별해 하나의 클러스터(cluster) 또는 비슷한 샘플의 그룹으로 할당하는 작업

In [ ]:
from sklearn.datasets import load_iris

In [ ]:
data = load_iris()
X = data.data
print(X.shape)

y = data.target
print(y.shape)

data.target_names

같은 데이터를 두고 레이블이 있을 때와 없을 때를 비교해보자.

In [ ]:
plt.figure(figsize=(9, 3.5))

plt.subplot(121)    # plt.subplot(1,2,1)과 같은 뜻
plt.plot(X[y==0, 2], X[y==0, 3], "yo", label="Iris setosa")
plt.plot(X[y==1, 2], X[y==1, 3], "bs", label="Iris versicolor")
plt.plot(X[y==2, 2], X[y==2, 3], "g^", label="Iris virginica")
plt.xlabel("Petal length", fontsize=14)
plt.ylabel("Petal width", fontsize=14)
plt.legend(fontsize=12)

plt.subplot(122)
plt.scatter(X[:, 2], X[:, 3], c="k", marker=".")
plt.xlabel("Petal length", fontsize=14)
plt.tick_params(labelleft=False)

save_fig("classification_vs_clustering_plot")
plt.show()

분석
<br> 
* 왼쪽 그림은 레이블이 된 데이터셋을 지도 학습 알고리즘으로 분류한 것이고
* 오른쪽 그림은 레이블이 없는 데이터를 비지도 학습 알고리즘으로 군집화한 결과라고 할 수 있다.

위에서는 특성 두개만을 가지고 분류, 군집화를 하였다.
<br> 이번에는 가우시안 혼합 모델을 사용하여 모든 특성을 사용하여 분류, 군집화를 해보자.

In [ ]:
from sklearn.mixture import GaussianMixture

In [ ]:
y_pred = GaussianMixture(n_components=3, random_state=42).fit(X).predict(X)
mapping_index = [np.argmax(np.bincount(y_pred[n:n+50])) for n in range(0, 150, 50)]
mapping = {mapping_index[i]:i for i in [0, 1, 2]}
y_pred = np.array([mapping[cluster_id] for cluster_id in y_pred])

In [ ]:
plt.plot(X[y_pred==0, 2], X[y_pred==0, 3], "yo", label="Cluster 1(setora)")
plt.plot(X[y_pred==1, 2], X[y_pred==1, 3], "bs", label="Cluster 2(versicolor)")
plt.plot(X[y_pred==2, 2], X[y_pred==2, 3], "g^", label="Cluster 3(virginica)")
plt.xlabel("Petal length", fontsize=14)
plt.ylabel("Petal width", fontsize=14)
plt.legend(loc="upper left", fontsize=12)
plt.show()

In [ ]:
np.sum(y_pred==y)

In [ ]:
len(y_pred)

In [ ]:
np.sum(y_pred==y) / len(y_pred)

분석
* 총 150개의 샘플 중에 145개를 올바르게 분류한 것을 알 수 있다.
* 정확도가 97%로 매우 높다.

군집이 사용되는 애플리케이션
* 고객 분류 - 추천 시스템
* 데이터 분석
* 차원 축소 기법 - 각 클러스터에 대한 친화성(affinity) 측정
* 이상치 탐지
* 준지도 학습
* 검색 엔진 - 이미지 찾기

##9.1.1 k-평균

> k-평균이란?<br>
반복 몇 번으로 레이블이 없는 데이터셋을 빠르고 효율적으로 클러스터로 묶을 수 있는 간단한 알고리즘

샘플 덩어리 다섯개로 이루어진 레이블이 없는 데이터 셋을 만들어보자.

In [ ]:
from sklearn.datasets import make_blobs

In [ ]:
blob_centers = np.array(
    [[ 0.2,  2.3],
     [-1.5 ,  2.3],
     [-2.8,  1.8],
     [-2.8,  2.8],
     [-2.8,  1.3]])
blob_std = np.array([0.4, 0.3, 0.1, 0.1, 0.1])

In [ ]:
X, y = make_blobs(n_samples=2000, centers=blob_centers,
                  cluster_std=blob_std, random_state=7)

In [ ]:
def plot_clusters(X, y=None):
    plt.scatter(X[:, 0], X[:, 1], c=y, s=1)
    plt.xlabel("$x_1$", fontsize=14)
    plt.ylabel("$x_2$", fontsize=14, rotation=0)

In [ ]:
plt.figure(figsize=(8, 4))
plot_clusters(X)
save_fig("blobs_plot")
plt.show()

이 데이터셋에 k-평균 알고리즘을 훈련해보자.
<br> **각 클러스터의 중심을 찾고 가장 가까운 클러스터에 샘플을 할당**한다.

In [ ]:
from sklearn.cluster import KMeans

k = 5   # 클러스트 개수 지정
        # 각 샘플은 다섯 개의 클러스터 중 하나에 할당됨. 
kmeans = KMeans(n_clusters=k, random_state=42)
y_pred = kmeans.fit_predict(X)

In [ ]:
y_pred              # 예측 레이블

In [ ]:
kmeans.labels_      # 실제 레이블

※ kmeans.labels_는 각 데이터가 어떤 클래스에 속하는지 결과를 도출한다.

In [ ]:
y_pred==kmeans.labels_

In [ ]:
y_pred is kmeans.labels_      # 예측 레이블과 실제 레이블의 모든 원소의 값이 같음을 나타냄

> 센트로이드(centroid)란?
<br> 샘플들이 모인 곳의 중심 (i.e. 클러스터의 중심)

In [ ]:
kmeans.cluster_centers_     # 알고리즘이 찾은 센트로이드 다섯 개
                            # 당연한거지만, 클러스트의 개수와 센트로이드의 개수는 같다.

In [ ]:
# 새로운 샘플에 가장 가까운 센트로이드의 클러스터를 할당
X_new = np.array([[0, 2], [3, 2], [-3, 3], [-3, 2.5]])
kmeans.predict(X_new)

※ **센트로이드와 각 샘플들의 거리가 최소**인 센트로이드의 클러스터를 할당한다.

클러스터의 결정 경계를 그려보자.

In [ ]:
# 클러스터의 결정 경계 그리기
def plot_data(X):
    plt.plot(X[:, 0], X[:, 1], 'k.', markersize=2)

def plot_centroids(centroids, weights=None, circle_color='w', cross_color='k'):
    if weights is not None:
        centroids = centroids[weights > weights.max() / 10]
    plt.scatter(centroids[:, 0], centroids[:, 1],
                marker='o', s=30, linewidths=8,
                color=circle_color, zorder=10, alpha=0.9)       # zorder => 레이어의 순서를 지정
                                                                # 숫자가 클수록 가장 바깥쪽(가장 눈에 잘 보이는 위치)
    plt.scatter(centroids[:, 0], centroids[:, 1],
                marker='x', s=50, 
                color=cross_color, zorder=11, alpha=1)          # zorder가 더 크기 때문에 센트로이드 X 표시가 샘플 표시보다 위에 표시됨

def plot_decision_boundaries(clusterer, X, resolution=1000, show_centroids=True,
                             show_xlabels=True, show_ylabels=True):
    mins = X.min(axis=1) - 0.1
    maxs = X.max(axis=1) + 0.1      # 모든 샘플들이 그림에 잘 보이게 하기 위해서 y축 범위를 최소/최대값보다 0.1씩 작고/크게 만듦.
    xx, yy = np.meshgrid(np.linspace(mins[0], maxs[0], resolution),     # resolution => # of samples to generate(min과 max 사이를 몇개로 쪼갤지 결정)
                         np.linspace(mins[1], maxs[1], resolution))
    Z = clusterer.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    plt.contourf(Z, extent=(mins[0], maxs[0], mins[1], maxs[1]),
                cmap="Pastel2")
    plt.contour(Z, extent=(mins[0], maxs[0], mins[1], maxs[1]),
                linewidths=1, colors='k')
    plot_data(X)
    if show_centroids:
        plot_centroids(clusterer.cluster_centers_)

    if show_xlabels:
        plt.xlabel("$x_1$", fontsize=14)
    else:
        plt.tick_params(labelbottom=False)
    if show_ylabels:
        plt.ylabel("$x_2$", fontsize=14, rotation=0)
    else:
        plt.tick_params(labelleft=False)

In [ ]:
plt.figure(figsize=(8, 4))
plot_decision_boundaries(kmeans, X)
save_fig("voronoi_plot")
plt.show()

클러스터의 결정 경계를 그리면 보로노이 다이어그램을 얻을 수 있다.
> 보로노이 다이어그램(Voronoi tessellation)이란?
<br> 평면을 특정 점까지의 거리가 가장 가까운 점의 집합으로 분할한 그림

k-평균 알고리즘은 샘플을 클러스터에 할당할 때 센트로이드까지의 거리를 고려하는게 전부이기 때문에, <br> 클러스터의 크기가 많이 다르면 잘 작동하지 않는다.

> 하드 군집(hard clustering) : 샘플을 하나의 클러스터에 할당하는 것
<br> 소프트 군집(soft clustering) : 클러스터마다 샘플에 점수를 부여하는 것

KMeans 클래스의 transform() 메서드는 샘플과 각 센트로이드 사이의 거리를 반환한다.

In [ ]:
kmeans.transform(X_new)     # 샘플 4개와 센트로이드 5개의 각각의 거리를 모두 반환

In [ ]:
np.linalg.norm(np.tile(X_new, (1, k)).reshape(-1, k, 2) - kmeans.cluster_centers_, axis=2)

In [ ]:
# 위의 tile() 함수를 알아보자
a = np.array([0, 1, 2])
b = np.tile(a, 2)      
d = np.tile(a,(1,2))
c = np.tile(a, (3, 2))

print(b)
print(d)
print(c)

np.tile(배열, 숫자) 함수는 array를 지정한 횟수만큼 반복한다.
<br> ※ 숫자 자리에 1차원이 들어가면 1차원으로 반환되고, 2차원이 들어가면 2차원으로 반환된다.

### k-평균 알고리즘